## 1. Install dependences and load data

In [ ]:
import PIL

In [ ]:
import matplotlib

In [ ]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import cv2
import imghdr

In [ ]:
# connect to drive
#from google.colab import drive
#drive.mount('/content/drive')

# Change to the directory with your dataset
%cd /content/drive/My Drive/ML_project

In [ ]:
#os.listdir()

In [ ]:
# Avoid OutOfMemory error | I do not have a GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
tf.keras.utils.image_dataset_from_directory??

In [ ]:
# Define the paths to the train and test directories
train_dir = 'data\\train'
test_dir = 'data\\test'

# with keras.utils we do not have to code the labels, and it preprocesses
train = tf.keras.utils.image_dataset_from_directory(train_dir, 
                                                    color_mode = 'grayscale')
test = tf.keras.utils.image_dataset_from_directory(test_dir, 
                                                   color_mode = 'grayscale')

# train and test are NOT preloaded, they are generators.
# we first convert into a numpy iterator


In [ ]:
# with iterators we can access the data pipeline
train_iterator = train.as_numpy_iterator()
test_iterator = test.as_numpy_iterator()

In [ ]:
# gets another batch from the iterator
train_batch = train_iterator.next()
# the batches have 2 variables, the first one is the image representation, the second is the label
print(len(train_batch))
train_batch[1]

In [ ]:
# Debug, why with the cell above Jupyter crashes?
print("Inspecting images and labels:")
for idx, img in enumerate(train_batch[0][:4]):
    print(f"Image {idx} - shape: {img.shape}, dtype: {img.dtype}, min: {img.min()}, max: {img.max()}")
    
print("Labels:", train_batch[1][:4])


## Data Processing
- Resize all images -> already done
- Bring everything to greyscale -> already done

In [ ]:
# shape of each image
print(train_batch[0].shape)

print(train_batch[0].min())
print(train_batch[0].max()) # we want values as small as possible -> /255
# this makes optimization more efficient -> we need to scale values

In [ ]:
# we implement the data transformation in the pipeline
# x represent images, y labels
train = train.map(lambda x,y: (x/255,y))

In [ ]:
train.as_numpy_iterator().next() # since we have shuffling data changes

In [ ]:
# check the values of the images
print(f"Min value of scaled images: {train.as_numpy_iterator().next()[0].min()}")
print(f"Max value of scaled images: {train.as_numpy_iterator().next()[0].max()}")

In [ ]:
# What is 0? What is 1? This makes jupyter crash
fig, ax = plt.subplots(ncols = 4, figsize = (20,20))
for idx, img in enumerate(train_batch[0][:4]):
    ax[idx].imshow(img)
    ax[idx].title.set_text(train_batch[1][idx])

## Build the Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout